In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

In [8]:
df = pd.read_csv("Zomato_Restaurants_All_Consolidated-Types.csv")
df = df.loc[(df["Sensor ID"] == 27) | (df["Sensor ID"] == 75) | (df["Sensor ID"] == 1),:]
df

,Unnamed: 0,Name,Aggregate Rating,Type,Price,Latitude,Longitude,Sensor ID,Sensor
49,80,Soul Cafe,3.0,Cafe,3,-37.813075,144.965281,1,Bourke Street Mall (North)
50,81,Song Tea,3.5,Beverages,1,-37.813810,144.965600,1,Bourke Street Mall (North)
51,82,Sensory Lab - Bourke Street,3.4,Cafe,2,-37.813398,144.964499,1,Bourke Street Mall (North)
52,83,SamSam,4.0,Restaurant,3,-37.812885,144.965210,1,Bourke Street Mall (North)
53,85,The Tasty Spicy Hot Pot,3.2,Restaurant,2,-37.812871,144.965395,1,Bourke Street Mall (North)
...,...,...,...,...,...,...,...,...,...
307,438,Café Verona,3.1,Cafe,2,-37.806243,144.958600,27,QV Market-Peel St
308,439,Don Camillo,3.7,Cafe,3,-37.805839,144.954280,27,QV Market-Peel St
326,471,Top Juice,3.4,Fast Food,2,-37.814532,144.972261,75,Spring St- Flinders st (West)
327,477,The Fair Trader,3.9,Cafe,3,-37.815551,144.972006,75,Spring St- Flinders st (West)


In [9]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>Type</dt><dd>{Type}</dd>
<dt>Rating</dt><dd>{Aggregate Rating}</dd>
<dt>Price</dt><dd>{Price}</dd>
<dt>Sensor</dt><dd>{Sensor}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
eatery_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Latitude", "Longitude"]]

In [11]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=eatery_info)

# Plot Heatmap
fig = gmaps.figure()

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))